In [31]:
import crepe
from scipy.io import wavfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [32]:
def zeropadding(array, SamplePerWindow):
    if(array.size%SamplePerWindow != 0):
        left = array.size%SamplePerWindow
        add = np.zeros(SamplePerWindow - left)
        new_arr = np.append(array, add)
        return new_arr
    else:
        return array

In [33]:
import os

X = []
SamplePerWindow = 480 ## sample rate 48000/s 0.01sec/window
for dirname, _, filenames in os.walk('new_dataset/'):    ## Read data
    for filename in filenames:
        sr, audio = wavfile.read(os.path.join(dirname,filename))
        new_audio = zeropadding(audio, SamplePerWindow)
        #print(os.path.join(dirname, filename), 'audio size: ', new_audio.size)
        audio_max = np.amax(new_audio)  #normalize data
        new_audio = new_audio / audio_max
        X = np.append(X, new_audio);
        
print('Total Training datasize:',X.size/480,' -frames')

Total Training datasize: 166753.0  -frames


In [34]:
Y=[]

for dirname, _, filenames in os.walk('txtfile/'):  ##Read the Y labels for the data
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        data= pd.read_csv(os.path.join(dirname, filename))
        temp = np.array(data.frequency)
        #print(os.path.join(dirname, filename), 'Frames:', temp.size)
        Y = np.append(Y,temp)

minY = np.amin(Y)
for i in range(Y.size):
    Y[i] = Y[i] - minY
maxY = np.amax(Y)
print(minY, maxY)
X = np.reshape(X, (Y.size-1,480,1))
Y = Y[0:Y.size-1]

print('Total labels:', Y.shape)
print('Total frames:', X.shape)

31.0 1951.0
Total labels: (166753,)
Total frames: (166753, 480, 1)


In [35]:
print('X data shape:',X.shape)
print('Y label shape:', Y.shape)
# for i in range(Y.size):
#     if Y[i] > 400:
#         Y[i] = 400
        
X_new = X[40000:120000]
Y_new = Y[40000:120000]

X data shape: (166753, 480, 1)
Y label shape: (166753,)


In [36]:
#split the data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_new,Y_new,test_size=0.2)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
SamplePerFrame = 480

(64000, 480, 1) (64000,)
(16000, 480, 1) (16000,)


In [49]:
#CNN Model
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout


model = Sequential()
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(480,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1024, activation='sigmoid'))
model.add(Dense(2048, activation='sigmoid'))
model.add(Dense(1952, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, 478, 128)          512       
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 239, 128)         0         
 1D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 237, 64)           24640     
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 118, 64)          0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 7552)              0         
                                                                 
 dense_6 (Dense)             (None, 1024)             

In [51]:
#Model Training
model.fit(x_train, y_train, epochs=5)
model.save('CNNModel')

Epoch 1/5
2000/2000 [==============================] - 147s 73ms/step - loss: 3.8585 - accuracy: 0.0785
Epoch 2/5
2000/2000 [==============================] - 142s 71ms/step - loss: 3.7826 - accuracy: 0.0895
Epoch 3/5
2000/2000 [==============================] - 145s 73ms/step - loss: 3.7069 - accuracy: 0.0988
Epoch 4/5
2000/2000 [==============================] - 158s 79ms/step - loss: 3.6242 - accuracy: 0.1080
Epoch 5/5
2000/2000 [==============================] - 160s 80ms/step - loss: 3.5412 - accuracy: 0.1210
INFO:tensorflow:Assets written to: CNNModel\assets


In [52]:
print('Testing data')
val_loss, val_acc = model.evaluate(x_test,y_test)

Testing data
500/500 [==============================] - 7s 14ms/step - loss: 4.1298 - accuracy: 0.0644


In [53]:
sr, audio = wavfile.read('test_dataset/test.wav')

new_audio = []

for i in range(audio.size):
    if audio[i]>60 or audio[i]<-60:
        new_audio = np.append(new_audio, audio[i])
new_audio = np.array(new_audio)
new_audio = zeropadding(new_audio, SamplePerWindow)

print(audio.size, new_audio.size)
dim = int(new_audio.size/480)

test_audio = np.reshape(new_audio, (dim, 480, 1))
print(test_audio.shape)
test_output = model.predict(test_audio)
print(test_output.shape)
output = []

for i in range(dim):
    #print('max:', np.argmax(test_output[i][1:300]))
    result = np.where(test_output[i][50:500] == np.amax(test_output[i][80:500]))   
    result = result
    #print(i, result)
    output = np.append(output, result) 
    
file1 = open("pitch_demo\CNNModel1_test.csv","w")
for i in range(dim):
    file1.write(str(output[i]))
    file1.write('\n')
file1.close()
    
print(output.shape)
print('pitch file prediction created')

111718 111840
(233, 480, 1)
(233, 1952)
(233,)
pitch file prediction created


In [54]:
sr, audio = wavfile.read('test_dataset/test1.wav')

new_audio = []

for i in range(audio.size):
    if audio[i]>60 or audio[i]<-60:
        new_audio = np.append(new_audio, audio[i])
new_audio = np.array(new_audio)
new_audio = zeropadding(new_audio, SamplePerWindow)

print(audio.size, new_audio.size)
dim = int(new_audio.size/480)

test_audio = np.reshape(new_audio, (dim, 480, 1))
print(test_audio.shape)
test_output = model.predict(test_audio)
print(test_output.shape)
output = []

for i in range(dim):
    #print('max:', np.argmax(test_output[i][1:300]))
    result = np.where(test_output[i][50:300] == np.amax(test_output[i][60:300]))   
    result = result
    #print(i, result)
    output = np.append(output, result) 
    
file1 = open("pitch_demo\CNNModel1_test1.csv","w")
for i in range(dim):
    file1.write(str(output[i]))
    file1.write('\n')
file1.close()
    
print(output.shape)

101393 101760
(212, 480, 1)
(212, 1952)
(212,)


In [21]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [22]:
#Another larger model

model = Sequential()
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(480,1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1952, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 478, 128)          512       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 239, 128)         0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 237, 128)          49280     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 118, 128)         0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 116, 256)          98560     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 58, 256)          0

In [23]:
#Model Training
batch_size = 32
steps_per_epoch = int( np.ceil(x_train.shape[0] / batch_size) )
model.fit(x_train, y_train, epochs=5, steps_per_epoch = steps_per_epoch)
model.save('CNN1Model')

Epoch 1/5
2000/2000 [==============================] - 234s 117ms/step - loss: 5.1438 - accuracy: 0.0152
Epoch 2/5
2000/2000 [==============================] - 217s 108ms/step - loss: 4.2705 - accuracy: 0.0469
Epoch 3/5
2000/2000 [==============================] - 242s 121ms/step - loss: 4.0204 - accuracy: 0.0664
Epoch 4/5
2000/2000 [==============================] - 221s 111ms/step - loss: 3.8784 - accuracy: 0.0786
Epoch 5/5
2000/2000 [==============================] - 237s 118ms/step - loss: 3.7450 - accuracy: 0.0925
INFO:tensorflow:Assets written to: CNN1Model\assets


In [39]:
print('Testing data')
val_loss, val_acc = model.evaluate(x_test,y_test)

Testing data
500/500 [==============================] - 5s 11ms/step - loss: 4.0450 - accuracy: 0.0625


In [29]:
sr, audio = wavfile.read('test_dataset/test.wav')

new_audio = []

for i in range(audio.size):
    if audio[i]>60 or audio[i]<-60:
        new_audio = np.append(new_audio, audio[i])
new_audio = np.array(new_audio)
new_audio = zeropadding(new_audio, SamplePerWindow)
max_new_audio = np.amax(new_audio)
new_audio = new_audio/max_new_audio

print(audio.size, new_audio.size)
dim = int(new_audio.size/480)

test_audio = np.reshape(new_audio, (dim, 480, 1))
print(test_audio.shape)
test_output = model.predict(test_audio)
print(test_output.shape)
output = []

for i in range(dim):
    #print('max:', np.argmax(test_output[i][1:300]))
    result = np.where(test_output[i][50:500] == np.amax(test_output[i][80:500]))   
    result = result
    #print(i, result)
    output = np.append(output, result) 
    
file1 = open("pitch_demo\CNNModel2_test.csv","w")
for i in range(dim):
    file1.write(str(output[i]))
    file1.write('\n')
file1.close()
    
print(output.shape)

111718 111840
(233, 480, 1)
(233, 1952)
(233,)


In [30]:
sr, audio = wavfile.read('test_dataset/test1.wav')
new_audio = []

for i in range(audio.size):
    if audio[i]>60 or audio[i]<-60:
        new_audio = np.append(new_audio, audio[i])
new_audio = np.array(new_audio)
new_audio = zeropadding(new_audio, SamplePerWindow)
max_new_audio = np.amax(new_audio)
new_audio = new_audio/max_new_audio

print(audio.size, new_audio.size)
dim = int(new_audio.size/480)

test_audio = np.reshape(new_audio, (dim, 480, 1))
print(test_audio.shape)
test_output = model.predict(test_audio)
print(test_output.shape)
output = []

for i in range(dim):
    #print('max:', np.argmax(test_output[i][1:300]))
    result = np.where(test_output[i][50:500] == np.amax(test_output[i][50:500]))   
    result = result
    #print(i, result)
    output = np.append(output, result) 
    
file1 = open("pitch_demo\CNNModel2_test1.csv","w")
for i in range(dim):
    file1.write(str(output[i]))
    file1.write('\n')
file1.close()
    
print(output.shape)

101393 101760
(212, 480, 1)
(212, 1952)
(212,)
